In [2]:
import numpy as np
import pandas as pd

import scipy
from statsmodels.stats.weightstats import *
from statsmodels.stats.proportion import proportion_confint

In [6]:
def z_stat(p1,p2,n1,n2):
    P = float(p1*n1 + p2*n2) / (n1 + n2)
    return (p1 - p2) / np.sqrt(P * (1 - P) * (1. / n1 + 1. / n2))

In [84]:
1 - scipy.stats.norm.cdf(z_stat(10/float(34),1/float(4),34,16))

0.37293045872523534

In [23]:
from matplotlib.colors import ListedColormap
from sklearn import cross_validation, datasets, linear_model, metrics

import numpy as np

In [32]:
log_regressor_1 = linear_model.LogisticRegression(random_state = 1)

In [33]:
data = pd.read_csv('banknotes.txt', sep = '\t')

In [34]:
data.shape

(200, 7)

In [35]:
X = data.loc[:, data.columns != 'real']

In [36]:
Y = data['real']

In [54]:
x_train,  x_test,y_train, y_test = cross_validation.train_test_split(X,Y, test_size = 0.25, random_state = 1)

In [55]:
x_train_1 = x_train[['X1', 'X2', 'X3']]

In [56]:
log_regressor_1.fit(x_train_1, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=1, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [86]:
log_regressor_2 = linear_model.LogisticRegression(random_state = 1)

In [87]:
x_train_2 = x_train[['X4', 'X5', 'X6']]

In [88]:
log_regressor_2.fit(x_train_2,y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=1, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [102]:
prediction_1 = log_regressor_1.predict(x_test[['X1','X2','X3']])

In [103]:
prediction_2 = log_regressor_2.predict(x_test[['X4','X5','X6']])

In [91]:
def proportions_diff_confint_rel(sample1, sample2, alpha = 0.05):
    z = scipy.stats.norm.ppf(1 - alpha / 2.)
    sample = zip(sample1, sample2)
    n = len(sample)
        
    f = sum([1 if (x[0] == 1 and x[1] == 0) else 0 for x in sample])
    g = sum([1 if (x[0] == 0 and x[1] == 1) else 0 for x in sample])
    
    left_boundary = float(f - g) / n  - z * np.sqrt(float((f + g)) / n**2 - float((f - g)**2) / n**3)
    right_boundary = float(f - g) / n  + z * np.sqrt(float((f + g)) / n**2 - float((f - g)**2) / n**3)
    return (left_boundary, right_boundary)

In [92]:
def proportions_diff_z_stat_rel(sample1, sample2):
    sample = zip(sample1, sample2)
    n = len(sample)
    
    f = sum([1 if (x[0] == 1 and x[1] == 0) else 0 for x in sample])
    g = sum([1 if (x[0] == 0 and x[1] == 1) else 0 for x in sample])
    
    return float(f - g) / np.sqrt(f + g - float((f - g)**2) / n )

In [96]:
def proportions_diff_z_test(z_stat, alternative = 'two-sided'):
    if alternative not in ('two-sided', 'less', 'greater'):
        raise ValueError("alternative not recognized\n"
                         "should be 'two-sided', 'less' or 'greater'")
    
    if alternative == 'two-sided':
        return 2 * (1 - scipy.stats.norm.cdf(np.abs(z_stat)))
    
    if alternative == 'less':
        return scipy.stats.norm.cdf(z_stat)

    if alternative == 'greater':
        return 1 - scipy.stats.norm.cdf(z_stat)

In [104]:
sum(abs(y_test - prediction_1))

10L

In [105]:
proportions_diff_z_test(proportions_diff_z_stat_rel(abs(y_test - prediction_1), abs(y_test - prediction_2)))

0.0032969384555543435

In [106]:
proportions_diff_confint_rel(abs(y_test - prediction_1), abs(y_test - prediction_2))

(0.059945206279614305, 0.3000547937203857)

In [85]:
1 - scipy.stats.norm.cdf(1.65)

0.0494714680336481

In [77]:
sum(y_test - prediction_1)

4L

58     0
40     0
34     0
102    0
184    0
198    0
95     0
4      0
29     0
168    0
171    0
18     0
11     0
89     0
110    1
118    0
159    0
35     0
136    0
59     0
51     0
16     0
44     0
94     0
31     0
162    0
38     0
28     0
193    0
27     0
47     0
165    0
194    0
177    0
176    0
97     0
174    0
73     0
69     0
172    0
108    0
107    0
189    0
14     0
56     0
19     0
114    0
39     0
185    0
124    0
Name: real, dtype: int64